# This section uses Magpie to calculate surface brightness profile

This is equivalently what the previous section does, but magpie is used to calculate surface brightness profile for each Ha Hb line maps and weights, averaged out over the polar angle phi, for each given pixel distance.

what does the package [magpie](https://github.com/knaidoo29/magpie/tree/master) does:

It transforms grid in Cartesian coordinates into (r,phi) polar coordinates, while preserving the size of the surface area of each pixel. Therefore one can calculate surface brightness of a certain ring area by summing over phi of a given radius range. This method is used in [Matharu 2023](https://iopscience.iop.org/article/10.3847/2041-8213/acd1db/pdf) and [Matharu 2024](https://arxiv.org/pdf/2404.17629).

In [2]:
import  magpie              as     magpie

In [3]:

import  numpy               as     np
from    astropy.table       import Table
from    astropy.io          import fits
from    astropy.cosmology   import Planck18
import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
from    matplotlib          import use
from    tqdm                import tqdm
from    concurrent.futures  import ThreadPoolExecutor, as_completed
import  sys, os
from    IPython.display     import clear_output



#this is just a handy little function to return the desired file path
#give one entry in the object list, return the desired file path
def file_name(obj,prefix,filetype='fits'):
    field = obj['subfield'].lower()
    id    = str(obj['ID']).zfill(5)
    return f"hlsp_clear_hst_wfc3_{field}-{id}_g102-g141_v4_{prefix}.{filetype}"

def find_data(name,hdu):
    for image in hdu:
        if name == image.name:
            return image

obj_lis = Table.read('obj_lis_selected.fits')

True

In [ ]:
import magpie.montecarlo

def spatial_remap(map,pixel_length):
    b2r = magpie.montecarlo.Box2Ring()
    b2r.setup_box(-25*pixel_length,25*pixel_length,50,
                -25*pixel_length,25*pixel_length,50)
    b2r.setup_polar_lin(0., 25*pixel_length, 12, 10, center=[0., 0.])
    b2r.get_weights() 
    return np.linspace(b2r.redges[0], b2r.redges[-1],12), b2r.remap(map)

def radial_profile(obj,linemap,weight,seg,pixel_length):
        linemap = linemap.data; weight = weight.data; seg = seg.data

        seg = np.where(seg==obj['ID'],1,0)
        seg = np.where(linemap>0,seg,0)
        seg = spatial_remap(seg,pixel_length)[1]
        seg_weight = seg/np.sum(seg,axis=0)

        r, linemap_r,   = spatial_remap(np.where(linemap>0,linemap,0),pixel_length)
        linemap_r_wht = spatial_remap(np.where(linemap>0,weight,0),pixel_length)[1]


        map_r   = np.average(linemap_r,weights=seg_weight,axis=0)
        map_r_err = 1/np.sum(linemap_r_wht*seg,axis=0)**0.5
        map_r_std = np.average((linemap_r-map_r)**2,weights = seg_weight,axis=0)**0.5
        map_r_err = (map_r_err**2 + map_r_std**2)**0.5
        return r, map_r, map_r_err 


def gen_radial_table(obj,LINE_HA='LINE_HA',LINE_HB='LINE_HB_CONV',
                    LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV'):
    #try:
    path = f"data_extracted/{file_name(obj,prefix='extracted')}"
    with fits.open(path,mode='update') as hdu:
        print(obj)
        
        pixel_length = np.deg2rad(find_data(LINE_HA,hdu).header['PIXASEC']/3600) * Planck18.angular_diameter_distance(hdu[1].header['Z_MAP']).to(u.kpc).value

        r, ha_r, ha_r_err = radial_profile(obj,find_data(LINE_HA,hdu),find_data(LINEWHT_HA,hdu),find_data('SEG',hdu),pixel_length)
        r, hb_r, hb_r_err = radial_profile(obj,find_data(LINE_HB,hdu),find_data(LINEWHT_HB,hdu),find_data('SEG',hdu),pixel_length)

        balmer_r     = ha_r/hb_r
        balmer_r_err = ((ha_r_err/hb_r)**2 + (hb_r_err**2 * (ha_r/hb_r**2)**2))**0.5
        
        cols = [
            fits.Column(name='DISTANCE [kpc]',                       format='E', array=r),
            fits.Column(name='Ha_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=ha_r),
            fits.Column(name='Ha_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=ha_r_err),
            fits.Column(name='Hb_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=hb_r),
            fits.Column(name='Hb_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=hb_r_err),
            fits.Column(name='BALMER_DECREM',                        format='E', array=balmer_r),
            fits.Column(name='BALMER_DECREM_ERR',                    format='E', array=balmer_r_err),
        ]

        #choose the right name for table
        if 'CONV' in LINE_HB:
            name_addon = '_CONV'
        else:
            name_addon = ''
        if 'BG' not in LINE_HA:
            name = f'RAD_PROFILE{name_addon}'
            new_table = fits.BinTableHDU.from_columns(cols, name=name)
        else:
            name = f'RAD_PROFILE{name_addon}_BG'
            new_table = fits.BinTableHDU.from_columns(cols, name=name)
        
        clear_output(wait=True)

        #save or update table
        for i,image in enumerate(hdu):
            if  image.name == name:
                hdu[i] = new_table
                hdu.flush()
                return f"{obj['subfield']}-{obj['ID']} processed"

        hdu.append(new_table)
        hdu.flush()
        return f"{obj['subfield']}-{obj['ID']} processed"
    #except Exception as e:
    #        return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"


def cat_process(obj_lis,max_threads=1):
        results = []
        if max_threads > 1 :
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(gen_radial_table,obj):obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in obj_lis:
                results.append(gen_radial_table(obj))
            return results
def main():

    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process(obj_lis,max_threads=6)

    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print(results)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)


if __name__ == '__main__':
    main()


Calculating weights : |#############################################_____|  90% 

Processing:  69%|██████▉   | 109/158 [02:04<01:01,  1.25s/it]

  ID          RA               DEC         nlines        z_50              z_02             z_16             z_84              z_97            z_MAP            z_RISK       ArIII-7138_FLUX   ArIII-7138_FLUX_ERR ArIII-7138_EW_RF_16 ArIII-7138_EW_RF_50 ArIII-7138_EW_RF_84 CIII-1908_FLUX CIII-1908_FLUX_ERR CIII-1908_EW_RF_16 CIII-1908_EW_RF_50 CIII-1908_EW_RF_84 CIV-1549_FLUX CIV-1549_FLUX_ERR CIV-1549_EW_RF_16 CIV-1549_EW_RF_50 CIV-1549_EW_RF_84 H8_FLUX H8_FLUX_ERR H8_EW_RF_16 H8_EW_RF_50 H8_EW_RF_84 H9_FLUX H9_FLUX_ERR H9_EW_RF_16 H9_EW_RF_50 H9_EW_RF_84      Ha_FLUX         Ha_FLUX_ERR       Ha_EW_RF_16       Ha_EW_RF_50       Ha_EW_RF_84        Hb_FLUX         Hb_FLUX_ERR        Hb_EW_RF_16       Hb_EW_RF_50       Hb_EW_RF_84        Hd_FLUX        Hd_FLUX_ERR       Hd_EW_RF_16       Hd_EW_RF_50       Hd_EW_RF_84    HeI-1083_FLUX HeI-1083_FLUX_ERR HeI-1083_EW_RF_16 HeI-1083_EW_RF_50 HeI-1083_EW_RF_84  HeI-5877_FLUX   HeI-5877_FLUX_ERR  HeI-5877_EW_RF_16 HeI-5877_EW_RF_50 HeI-5877_EW_RF

In [3]:
%matplotlib inline
def plot_balmer_decrem(obj):
    try:
        path = f"data_extracted/{file_name(obj,prefix='extracted')}"
        with fits.open(path) as hdu:    
            
            pixel_length = np.deg2rad(find_data('LINE_HA',hdu).header['PIXASEC']/3600) * Planck18.angular_diameter_distance(hdu[1].header['Z_MAP']).to(u.kpc).value
        
            ax = plt.figure(figsize=(8,10));i=1
            
            for index in [3,2,4,6]:
                ax.add_subplot(int(f'32{i}'));i+=1
                plt.imshow(hdu[index].data,#np.where(seg,hdu[index].data,0),
                            norm=colors.Normalize(vmin=0),
                            origin='lower',
                            cmap = 'plasma_r')
                plt.plot([3,7],[4,4])
                plt.text(5,5,f'{round(pixel_length*4,2)}kpc')
                plt.title(f"{hdu[index].name}_{obj['subfield']}_{obj['ID']}{obj['tag']}")
                plt.colorbar()
            
            r,ha_r,ha_r_err, hb_r, hb_r_err, balmer_r, balmer_r_err = np.vstack(find_data('RAD_PROFILE',hdu).data).transpose()
            r,ha_r_bg,ha_r_err_bg, hb_r_bg, hb_r_err_bg, balmer_r_bg, balmer_r_err_bg = np.vstack(find_data('RAD_PROFILE_BG',hdu).data).transpose()
            
            ax.add_subplot(325)
            plt.errorbar(r,ha_r,yerr = ha_r_err,fmt='bo:',label='Ha')
            plt.errorbar(r,hb_r,yerr = hb_r_err,fmt='go:',label='Hb')
            plt.errorbar(r,ha_r_bg,yerr = ha_r_err_bg,fmt='bo:',label='Ha_bg_subtracted',alpha=0.4)
            plt.errorbar(r,hb_r_bg,yerr = hb_r_err_bg,fmt='go:',label='Hb_bg_subtracted',alpha=0.4)
            plt.xlabel('distance [kpc]'); plt.ylabel('flux [1e-7 erg/s/cm2]')
            plt.yscale('log');plt.grid();plt.legend()
            
            ax.add_subplot(326)
            plt.errorbar(r,balmer_r,yerr = balmer_r_err,fmt='ro:',label='balmer_decrem')
            plt.errorbar(r,balmer_r_bg,yerr = balmer_r_err_bg,fmt='bo:',label='balmer_decrem_bg_subtracted',alpha=0.4)
            plt.xlabel('distance [kpc]');plt.ylabel('Ha/Hb')
            plt.xlim(0,8);plt.ylim(-5,10)
            plt.grid();plt.legend()

            plt.savefig(f"radial_balmer_decrem/{obj['subfield']}-{obj['ID']}.png")
            if obj['sn_hb'] >10:
                plt.savefig(f"sn_10/{obj['subfield']}-{obj['ID']}.png")
            plt.close('all')
            return f"{obj['subfield']}-{obj['ID']}saved"
        

    except Exception as e:
            return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"

def cat_process(obj_lis,max_threads=1):
        results = []
        if max_threads>1:
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(plot_balmer_decrem,obj):obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in tqdm(obj_lis):
                results.append(plot_balmer_decrem(obj))
            return results


if __name__ == '__main__':
    use('Agg')
    obj_lis = Table.read('obj_lis_selected.fits')
    results = cat_process(obj_lis,max_threads=1)
    number = 0
    for result in results:
        if 'error' in result:
            number +=1
            print(result)
    print('total number of obj processed:',len(results))
    print('number of failed obj',number)

    

/tmp/ipykernel_20856/885594253.py:65: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed in 3.10.  To suppress this warning, explicitly call plt.close('all') first.
  use('Agg')
100%|██████████| 158/158 [01:52<00:00,  1.41it/s]

total number of obj processed: 158
number of failed obj 0
